In [ ]:
#### MUST always run twice, alternating the settings of p_immune and p_alt_immune !######

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

#sns.set()

max_population = 10 * 10 # must be square of a number
p_propagation = 0.01 # risk of contamination when meeting an infected,
# very sensitive!!! setting it to 0.03 tops at 50.000 infected,
# setting it at 0.02 the infection dies out after the only infected person
# after 4 time units no longer carries the virus

p_immune = 0.3
p_alt_immune = 0.1

p_infected = 0.05 #0.01 # ratio infected in initial population
ttl = 8 # time a person remains infected
nr_contacts = 20 # nr of contacts per unit time (iteration)
iterations = 25 # must be a square,subplots & rows,cols depend on this

img = np.zeros( (
        np.sqrt(max_population).astype(int),
        np.sqrt(max_population).astype(int))).astype(int)

np.random.seed(4711)


In [ ]:
def set_pixel(pid,value):
        global img
        
        row = pid // np.sqrt(max_population).astype(int)
        col = pid % np.sqrt(max_population).astype(int)
        img[row,col] = value
        return (row,col)

class Person:
        #contacts_matrix = np.eye(int(max_population)).astype(bool)
        #np.fill_diagonal(contacts_matrix,0)

        infected = np.zeros(max_population).astype(bool)
        immune = np.zeros(max_population).astype(bool)
                
        def make_contact(p1,p2):

                p1.add_friend(p2)
                p2.add_friend(p1)
                
                if p1.infected:
                        propagate = np.random.choice([False,True],
                                p=[ 1 - p_propagation,p_propagation ])
                        
                        if not p2.infected and not p2.immune and p1.ttl > 0:
                                p2.infected = propagate
                                Person.infected[p2.pid] = propagate
  
                if p2.infected:
                        propagate = np.random.choice([False,True],
                                p=[ 1 - p_propagation,p_propagation ])
                        
                        if not p1.infected and not p1.immune and p2.ttl > 0:
                                p1.infected = propagate
                                Person.infected[p1.pid] = propagate
                        
                #Person.contacts_matrix[p1.pid,p2.pid] = True
                #Person.contacts_matrix[p2.pid,p1.pid] = True
                
        def __init__(self, pid,name,ttl,infected=False,immune=False):
                self.pid = pid
                self.name = name
                self.ttl = ttl
                self.infected = infected
                self.immune = immune
                self.contacts = []
                
        def show(self):
                print ('pid:',self.pid)
                print('name:',self.name)
                print('time to live:',self.ttl)
                print ('infected:',self.infected)
                print ('nr contacts:',len(self.contacts))
                
        def add_friend(self,person):
                self.contacts.append(person)
                
        def get_contacts(self):
                return self.contacts
        
        def del_friend(self,pid):
                self.remove(pid)

def create_population(size,ttl=10):
        pop_list = []
        for p in range(size):
                infected = np.random.choice([False,True],
                                p=[ 1 - p_infected,p_infected ])
                
                immune = np.random.choice([False,True],
                                         p = [ 1- p_immune,p_immune])
                
                P = Person(p,str(p),ttl,infected,immune)
                pop_list.append(P)
                Person.infected[p] = infected
                Person.immune[p] = immune
                
                if infected:
                    set_pixel(P.pid,100)
                if immune:
                    P.infected = False
                    set_pixel(P.pid,10)
                
        return pop_list

def create_contacts(population,nr_contacts=10):
        
        for p in population:
                        
                for f in range(nr_contacts):
                        friend_pid = np.random.choice(max_population-1)
                        if friend_pid != p.pid:
                                Person.make_contact(p,population[friend_pid])
                                
                
                    
                if p.infected:
                        p.ttl -= 1
                        set_pixel(p.pid,100)
                        
                        if p.ttl <= 0:
                                p.infected = False
                                p.immune = True
                                set_pixel(p.pid,10)
                                Person.infected[p.pid] = False
                                Person.immune[p.pid] = True
                                
                if p.immune:
                    set_pixel(p.pid,10)


In [ ]:
pop_list = create_population(max_population,ttl)

case_list = np.zeros((iterations,2)).astype(int)
case_list[0,0] = Person.infected.sum()
case_list[0,1] = nr_contacts

rows = np.sqrt(iterations).astype(int)
cols = np.sqrt(iterations).astype(int)

fig, axes = plt.subplots(nrows=rows,ncols=cols,figsize=(18,12))

plt.style.use('classic')

for i in range(iterations):
        #print (Person.infected)
        cases = Person.infected.sum()
        immunes = Person.immune.sum()
        
        r = i // rows
        c = i % rows

        print ('iteration:',i, 'total infected:',Person.infected.sum(), 'total_immune:',Person.immune.sum())
        axes[r,c].matshow(img,cmap='viridis_r')
        msg = '{} Inf. {} Imm. , iter: {}'.format(
                Person.infected.sum(),Person.immune.sum(),i)
        
        axes[r,c].set_title(msg,fontdict={'fontsize':10})
        axes[r,c].tick_params(axis='both',which='both',bottom=False,top=False,
                              labelbottom=False)
        
        axes[r,c].set_xticklabels(labels=[])
        axes[r,c].set_yticklabels(labels=[])
        
        axes[r,c].grid(False)
        
        case_list[i,1] = immunes
        case_list[i,0] = cases
        
        if i < (iterations - 1):
            create_contacts(pop_list,nr_contacts)
        

        print ()

print (case_list)        

data = pd.DataFrame(case_list,columns=['data','immune'])
print (data)

plt.tight_layout()
plt.savefig('corona_graph_immune_{:.1f}.jpg'.format(p_immune),format='jpg')
data.to_pickle('corona.pkl')


In [ ]:
df = pd.read_pickle('corona.pkl')

decease_factor = 0.02
intensive_care = 0.15

# both below constants from corona.py
contacts = nr_contacts
population = max_population

df = df.rename(columns = {'data':'infected_total'})

df['new_per_day'] = df['infected_total'] - df['infected_total'].shift()
df['new_per_day'] = df['new_per_day'].apply(lambda x : x if x >= 0 else 0)

df.iloc[0,1] = df.iloc[0,0]

def dead_p_day(x):
    dead =  x * decease_factor
    if dead > 0:
        return np.ceil(dead).astype(int)
    return 0

df['dead_per_day'] = df['new_per_day'].apply(dead_p_day)
df['cumdead'] = df['dead_per_day'].cumsum()
df['hospitalized_p_day'] = (df['new_per_day'] * 0.15).astype(int)
df['hospitalized_cum'] = df['hospitalized_p_day'].cumsum()
df['R0'] = (df['new_per_day'] / df['infected_total'].shift()).shift(-1)
df['ratio'] = df['new_per_day'] / df['infected_total']
print (df.head(50))
df.to_pickle ('corona_post_plot.pkl')

print (df[['new_per_day','dead_per_day']].sum())
    
sns.set()

plt.figure(figsize=(18,12))
plt.title('Corona Sim : Nr of Infected of {}, number of contacts per time unit = {}'.format(population,contacts))
plt.bar (df.index,df.infected_total)
plt.xlabel('time units')
plt.ylabel('Nr infected')
_=plt.savefig('corona_{:.1f}.jpg'.format(p_immune),format='jpg')

'''
plt.figure(figsize=(18,12))
ax = plt.gca()
ax.set_yscale('log')
plt.title('Number of people infected,hospitalized & deceased per day {} contacts per w,  mortality rate of 2%'.format(contacts))
ax.plot(df.index,df['hospitalized_p_day'],label='hostpitalized per day',color='y')
ax.plot(df.index,df['new_per_day'],label = 'infected per day',color='g')
ax.plot(df.index,df['dead_per_day'],label='dead per day',color='r')
plt.legend(loc='upper left')
plt.savefig('corona_dead.jpg',format='jpg')

plt.figure(figsize=(18,12))
ax = plt.gca()
ax.set_yscale('log')
plt.title('Number of people infected,hospitalized & dead, in total')
ax.plot (df.index,df.infected_total,label='infected cumulative',color='g')
ax.plot(df.index,df['hospitalized_cum'],label='hospitalized cumulative',color='y')
ax.plot(df.index,np.cumsum(df['dead_per_day']),label='dead cumulative',color='r')
ax.legend(loc='upper left')
'''
df.iloc[:,:2].to_pickle('corona_sim_stats_{:.1f}.pkl'.format(p_immune))

In [ ]:
df['new_per_day'].plot(kind='bar',figsize=(18,12),title='Corona Sim : New cases')
plt.ylabel('Nr new cases')
plt.xlabel('time units')
plt.savefig('corona_sim_new_cases_{:.1f}.jpg'.format(p_immune),format='jpg')


In [ ]:
stat_df_no_immune = pd.read_pickle('corona_sim_stats_{:.1f}.pkl'.format(p_immune))
stat_df_immune = pd.read_pickle('corona_sim_stats_{:.1f}.pkl'.format(p_alt_immune))

print (stat_df_no_immune)
print (stat_df_immune)

In [ ]:
stat_df = stat_df_no_immune.join(stat_df_immune,lsuffix='{:.1f}'.format(p_immune),rsuffix='{:.1f}'.format(p_alt_immune))

stat_df.columns = [['infected_immunity_{:.1f}'.format(p_immune),
                    'immune{:.1f}'.format(p_immune),'infected_immunity_{:.1f}'.format(p_alt_immune),
                   'immune{:.1f}'.format(p_alt_immune)]]

stat_df


In [ ]:
stat_df[['infected_immunity_{:.1f}'.format(p_immune),'infected_immunity_{:.1f}'.format(p_alt_immune)]].plot(figsize=(18,12),
                                              title='Corona Sim : total infected per unit time, with herd immunity set at {:.1f} & {:.1f} %'.format(
                                             p_immune,p_alt_immune)) 
plt.ylabel('nr infected per unit time')
plt.xlabel('unit time')

plt.savefig('corona_sim_plot.jpg',format='jpg')